In [30]:
import pandas as pd
import numpy as np
import csv
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from pythainlp.corpus import stopwords
from pythainlp.tokenize import word_tokenize
import re
from sklearn.metrics import recall_score, precision_score, f1_score

#i=0
def tokenizer(tweet):
    #global i
    #i = i+1
    #if i%500==0: print(i)
    pattern = re.compile(r"[^\u0E00-\u0E4C]|^'|'$|''|\u0E46|\u0E3F")
    char_to_remove = re.findall(pattern, tweet)
    list_with_char_removed = [char for char in tweet if not char in char_to_remove]
    result_string = ''.join(list_with_char_removed)
    return word_tokenize(result_string, engine='deepcut')
        
#filename = "./dataset/forCLFnohash1label1000.csv"
filename = "./dataset/forCLFnohash1label4000.csv"

df = pd.read_csv(filename)
X = np.array(df['tweet'])
y = np.array(df['label'])


for i in range(len(X)):
    if type(X[i]) == float:
        print(i,"error!!!")
        X[i]=""
    #if i%100==0: print(i)
    
## Extract features from a tweet with TfIdf
v = TfidfVectorizer(tokenizer=tokenizer, stop_words=stopwords.words('thai'))
train_data_feature = v.fit_transform(X).toarray()
print(v.get_feature_names())
        
X = train_data_feature

3638 error!!!
['', 'ก', 'กก', 'กกก', 'กกกก', 'กกกกก', 'กกกกกก', 'กกกกกกก', 'กกกกกกกกกก', 'กกกกกกกกกกก', 'กกกกกกกกกกกก', 'กกกกกกกกกกกกกกกกกก', 'กกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกกก', 'กกกกกกกกกนั่น', 'กกกกบีบีเปีย', 'กกกกแมวช้าน', 'กกกคุณชิณ', 'กกกจิง', 'กกกอิไต้หวั', 'กกกฮือ', 'กกกุ', 'กกถึงแม้', 'กกผ่า', 'กฎ', 'กด', 'กดบัตร', 'กดสแปมนา', 'กดไลค์ไลค์หาสวรรค์วิมานหรอสั', 'กทม', 'กน', 'กรนนนนนนนนนน', 'กรนโว้ยยยยยยย', 'กรม', 'กรมชล', 'กรมเลย', 'กรรม', 'กรรมแม่ง', 'กรอ', 'กรอกหู', 'กรอบ', 'กระจก', 'กระจอ', 'กระจ่าง', 'กระชาก', 'กระดาก', 'กระดาน', 'กระดาษ', 'กระดิก', 'กระดุม', 'กระด้า', 'กระตุก', 'กระตุ้น', 'กระต่าย', 'กระทง', 'กระทบ', 'กระทะ', 'กระทันกัน', 'กระทันหัน', 'กระทืบ', 'กระทู้', 'กระทู้', 'กระปุก', 'กระหาย', 'กระอ๊าค', 'กระเด็น', 'กระเท', 'กระเทย', 'กระเป๋า', 'กระเพรา', 'กระเเส', 'กระแดะ', 'กระแด่ะ', 'กระแต', 'กระแตอาร์ส', 'กระแทก', 'กระแนะกระแหน', 'กระแหนะกระแหน', 'กระโดด', 'กระโปรง', 'กริฟฟินดอร์อี', 'กริ๊ด', 'กรีดร้อง', 'กรี้ด', 'กรี้ดมัน'

In [43]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
def train_model(X,y,n,C):
    features = X
    label =y
    
    R = 0.0
    P = 0.0
    F = 0.0
    C = np.zeros(64)
    
    skf = StratifiedKFold(n_splits=n, random_state=30, shuffle=True)
    
    for train_index, test_index in skf.split(features, label):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = label[train_index], label[test_index]
        tclf = svm.LinearSVC(class_weight='balanced', C=C, random_state=0).fit(X_train,y_train)
        predict = tclf.predict(X_test)
        
        r=recall_score(y_test,predict,average='micro')
        p=precision_score(y_test,predict,average='micro')
        f=f1_score(y_test,predict,average='micro')
        cm=confusion_matrix(y_test,predict)
        
        R = R + r
        P = P + p
        F = F + f
        C = C + cm
        
        print('Prediction')
        print(predict)
        print('y')
        print(y_test)
        print('scores: recall, precision, f1')
        print(r,p,f)
        print('confusion matrix')
        print(cm)
        print()
        
    print()
    print('Average')
    print(R/n,P/n,F/n)
    print(C)

In [44]:
train_model(X,y,5,1)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [56]:
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=0)
tclf = svm.LinearSVC(class_weight='balanced', C=1, random_state=0).fit(X_train,y_train)
predict = tclf.predict(X_test)
        
r=recall_score(y_test,predict,average=None)
p=precision_score(y_test,predict,average=None)
f=f1_score(y_test,predict,average=None)
cm=confusion_matrix(y_test,predict)
print(predict)
print(y_test)
print(r,p,f)
print(cm)

[6 8 6 5 1 2 7 4 3 3 4 4 4 5 7 7 6 6 8 4 2 3 2 8 2 5 7 7 1 1 8 7 3 7 5 7 4
 8 8 1 7 4 1 7 8 6 3 4 8 4 8 2 7 7 3 3 5 5 6 4 2 8 6 7 6 5 2 7 7 6 6 5 8 7
 6 4 5 8 3 8 3 3 1 8 1 3 5 7 3 3 7 6 1 4 5 4 7 3 8 2 6 7 1 7 8 4 6 8 3 2 1
 4 5 6 4 8 3 3 8 2 5 6 1 7 4 2 7 8 6 1 3 3 6 6 2 3 4 7 2 2 4 1 2 1 3 6 2 2
 6 7 1 5 4 2 3 8 7 6 2 5 1 3 8 1 4 1 1 1 7 7 8 6 2 8 7 6 1 2 7 2 6 1 6 5 5
 8 3 2 5 6 7 3 5 5 5 7 3 7 5 1 2 1 2 2 1 8 8 6 5 2 1 1 1 3 4 3 7 5 6 2 8 5
 7 7 5 3 3 6 3 7 1 3 7 3 7 7 7 5 5 3 6 7 7 3 6 8 2 3 5 5 7 7 7 2 4 3 1 5 4
 4 6 7 6 4 4 7 7 7 1 6 3 7 1 6 5 8 4 6 4 6 8 5 3 5 5 4 1 7 6 4 1 4 7 7 3 3
 5 5 5 7 2 8 3 5 6 5 4 8 5 6 6 1 4 4 7 7 3 1 1 7 4 8 4 2 6 4 3 8 2 7 5 8 8
 7 2 3 5 5 5 1 3 5 4 4 3 6 5 6 4 4 2 8 1 8 2 8 1 7 6 4 7 2 4 1 6 5 5 7 3 2
 8 1 8 5 7 3 1 2 7 6 1 3 1 4 4 7 2 7 5 6 3 6 4 2 7 6 5 7 8 4 2 2 2 3 5 7 8
 8 1 4 7 1 8 6 4 2 8 1 1 2 4 5 3 2 3 7 7 1 8 3 8 2 7 5 8 3 2 6 5 1 4 7 6 2
 4 5 2 7 6 4 2 7 8 1 2 2 1 1 5 1 3 6 8 3 5 1 7 7 3 6 2 7 2 1 4 5 8 7 5 8 7
 7 8 3 8 5 6 7 2 8 2 1 7 

In [52]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [54]:
# Plot non-normalized confusion matrix
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cm, classes=['1','2','3','4','5','6','7','8'],
                      title='Confusion matrix, without normalization')

Confusion matrix, without normalization
[[22 10 19  9  7 12 14  9]
 [ 5 39  8  7 18 14 10 16]
 [14  7 38  3  6 15 12  6]
 [ 6  6  5 32  7  8 16  7]
 [ 9  8  6  8 37 18 12  1]
 [11  8  9 11  7 18 15 12]
 [11  7  2 11  9  8 43  5]
 [13 11 13  3  8 10 15 34]]
